# Representing Assembly and Transformation Activities in Build Compilation

### Testing adding assembly activity and linking to composites

In [1]:
import sbol2

from sbol2build import (
    translate_abstract_to_plasmids,
    golden_gate_assembly_plan,
    rebase_restriction_enzyme,
    copy_sequences,
)

plasmid_collection = sbol2.Document()
sbh = sbol2.PartShop("https://synbiohub.org")
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/CIDARMoCloPlasmidsKit_collection/1/b7fdc21c6601a61d3166073a9e50f2c3843e1df5/share",
    plasmid_collection,
)
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloParts/CIDARMoCloParts_collection/1/038feff5f9c6c9b35cc95586ed6b14ec9bd3fea5/share",
    plasmid_collection,
)

abstract_design_doc = sbol2.Document()
abstract_design_doc.read("tests/test_files/moclo_parts_circuit.xml")

backbone_doc = sbol2.Document()
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/DVK_AE/1/647c5b2458567dcce6b0a37178d352b8ffa9a7fe/share",
    backbone_doc,
)

mocloplasmid_list = translate_abstract_to_plasmids(
    abstract_design_doc, plasmid_collection, backbone_doc
)

# run assembly
part_documents = []
restriction_enzyme = rebase_restriction_enzyme("BsaI")
assembly_plan = sbol2.ModuleDefinition()

for mocloPlasmid in mocloplasmid_list:
    new_doc = sbol2.Document()
    mocloPlasmid.definition.copy(new_doc)
    copy_sequences(mocloPlasmid.definition, new_doc, plasmid_collection)
    part_documents.append(new_doc)

assembly_doc = sbol2.Document()
assembly_obj = golden_gate_assembly_plan(
    "assembly_plan", part_documents, backbone_doc, "BsaI", assembly_doc
)

assembly_obj.run()

found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_AB/1 with ['Fusion_Site_B', 'Fusion_Site_A']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_EB/1 with ['Fusion_Site_B', 'Fusion_Site_E']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_FB/1 with ['Fusion_Site_B', 'Fusion_Site_F']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_GB/1 with ['Fusion_Site_G', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/B0034/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0034_BC/1 with ['Fusion_Site_B', 'Fusion_Site_C']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/E0030_yfp/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0030_CD/1 with ['F

[[<sbol2.componentdefinition.ComponentDefinition at 0x10c2b7770>,

In [2]:
assembly_doc.write("assembly_activity.xml")

'Valid.'